In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
# Load the recycled dataset
train_data = np.load('./dataset/recycled_32_train.npz')
test_data = np.load('./dataset/recycled_32_test.npz')

x_train = train_data['x']
y_train = train_data['y']
x_test = test_data['x']
y_test = test_data['y']

# Preprocess the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape the data
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)

x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

In [10]:
print(x_train.shape)
print(x_test.shape)

(10000, 32, 32, 3)
(1500, 32, 32, 3)


In [5]:
dev = qml.device("default.qubit", wires=4)
# Random circuit parameters
rand_params = np.random.uniform(high=2 * np.pi, size=(1, 4))

@qml.qnode(dev, interface="autograd")
def circuit(phi):
    # Encoding of 4 classical input values
    for j in range(4):
        qml.RY(np.pi * phi[j], wires=j)

    # Random quantum circuit
    RandomLayers(rand_params, wires=list(range(4)))

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]

In [7]:
def quanv(image):
    """Convolves the input image with many applications of the same quantum circuit."""
    out = np.zeros((16, 16, 4))

    # Loop over the coordinates of the top-left pixel of 2X2 squares
    for j in range(0, 32, 2):
        for k in range(0, 32, 2):
            # Process a squared 2x2 region of the image with a quantum circuit
            q_results = circuit(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ]
            )
            # Assign expectation values to different channels of the output pixel (j/2, k/2)
            for c in range(4):
                out[j // 2, k // 2, c] = q_results[c]
    return out

In [11]:
SAVE_PATH = "quanvolution/"
PREPROCESS = False

if PREPROCESS:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(x_train):
        print("{}/{}        ".format(idx + 1, 10000), end="\r")
        q_train_images.append(quanv(img))
    q_train_images = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(x_test):
        print("{}/{}        ".format(idx + 1, 1500), end="\r")
        q_test_images.append(quanv(img))
    q_test_images = np.asarray(q_test_images)

    # Save pre-processed images
    np.save(SAVE_PATH + "q_train_images.npy", q_train_images)
    np.save(SAVE_PATH + "q_test_images.npy", q_test_images)


# Load pre-processed images
q_train_images = np.load(SAVE_PATH + "q_train_images.npy")
q_test_images = np.load(SAVE_PATH + "q_test_images.npy")

Quantum pre-processing of train images:
10000/10000        
Quantum pre-processing of test images:


NameError: name 'SAVE_PATH' is not defined

In [4]:
# Compile the model
def MyModel():
    """Initializes and returns a custom Keras model
    which is ready to be trained."""
    model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [8]:
# Step 2: Training
q_model = MyModel()

q_history = q_model.fit(
    q_train_images,
    y_train,
    validation_data=(q_test_images, y_test),
    batch_size=32,
    epochs=30,
    verbose=2,
)

Epoch 1/30
313/313 - 0s - loss: 1.2644 - accuracy: 0.4940 - val_loss: 1.1593 - val_accuracy: 0.5247 - 214ms/epoch - 684us/step
Epoch 2/30
313/313 - 0s - loss: 1.0947 - accuracy: 0.5792 - val_loss: 1.1026 - val_accuracy: 0.5547 - 100ms/epoch - 320us/step
Epoch 3/30
313/313 - 0s - loss: 1.0497 - accuracy: 0.5979 - val_loss: 1.0910 - val_accuracy: 0.5633 - 100ms/epoch - 318us/step
Epoch 4/30
313/313 - 0s - loss: 1.0226 - accuracy: 0.6051 - val_loss: 1.0630 - val_accuracy: 0.5707 - 99ms/epoch - 317us/step
Epoch 5/30
313/313 - 0s - loss: 0.9974 - accuracy: 0.6152 - val_loss: 1.0447 - val_accuracy: 0.5780 - 100ms/epoch - 319us/step
Epoch 6/30
313/313 - 0s - loss: 0.9784 - accuracy: 0.6231 - val_loss: 1.0374 - val_accuracy: 0.5827 - 100ms/epoch - 321us/step
Epoch 7/30
313/313 - 0s - loss: 0.9651 - accuracy: 0.6295 - val_loss: 1.0308 - val_accuracy: 0.5780 - 99ms/epoch - 317us/step
Epoch 8/30
313/313 - 0s - loss: 0.9524 - accuracy: 0.6300 - val_loss: 1.0182 - val_accuracy: 0.5873 - 98ms/epoch 

In [9]:
# Step 3: Evaluation
loss, accuracy = q_model.evaluate(q_test_images, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

47/47 [==============================] - 0s 403us/step - loss: 0.9496 - accuracy: 0.6087
Loss: 0.949603259563446
Accuracy: 0.6086666584014893
